In [2]:
import pandas as pd
import numpy as np

In [3]:
#CLV=pd.read_excel('C:/Users/miavila/Desktop/Copia de CLV 16-12_31-05.xlsx',sheet_name='JULIO')

In [527]:
impala=pd.read_excel('C:/Users/miavila/Desktop/CRUCE SCORE/2. ENGAGE/INT/query-impala-2922005.xlsx')
ATE=pd.read_excel('C:/Users/miavila/Desktop/CRUCE SCORE/2. ENGAGE/INT/Copia de Reporte cc engage Agosto.xlsx',sheet_name='Atendidas')
ABA=pd.read_excel('C:/Users/miavila/Desktop/CRUCE SCORE/2. ENGAGE/INT/Copia de Reporte cc engage Agosto.xlsx',sheet_name='Abandonadas')

In [528]:
ATE.columns, ABA.columns

(Index(['Cedula', 'Nombres', 'Telefono', 'Grupo Tratamiento', 'Connid',
        'Fecha2', 'HoraSegs', 'HH', 'MM', 'SS', 'Hora'],
       dtype='object'),
 Index(['FechaDia', 'Hora', 'Nit', 'Empresa', 'Telefono', 'Segmentos',
        'Segmento Global', 'Origen', 'Connid'],
       dtype='object'))

In [529]:
ABA.head()

FechaDia      Hora           Nit                         Empresa  \
0 2022-08-01  06:25:16  7.943499e+07  CARLOS HUMBERTO HERRON ALVAREZ   
1 2022-08-01  06:29:17  4.167453e+07           ZORAIDA HERRERA FERRO   
2 2022-08-01  06:39:02  8.608466e+07    DIEGO EFRAIN CASTELL TIVIDOR   
3 2022-08-01  06:42:14  1.097388e+09        ALEJANDRO GUZMAN COHECHA   
4 2022-08-01  06:44:47  3.943961e+07        FLOR MARIA SILVA ORDONEZ   

       Telefono                Segmentos Segmento Global     Origen  \
0  3144145098.0  NARANJA VENTA >1000.000         NARANJA  COMERCIAL   
1  3108578242.0  NARANJA VENTA >1000.000         NARANJA  COMERCIAL   
2  3124263050.0          BLANCO COBRANZA          BLANCO  COMERCIAL   
3  3163415160.0                SIN COLOR       SIN COLOR  COMERCIAL   
4  3128762373.0                SIN COLOR       SIN COLOR  COMERCIAL   

             Connid  
0  019a0338ff79fcf5  
1  019a0338ff79fcf9  
2  01d403393d05dccf  
3  019a0338ff79fd21  
4  01d403393d05dcdd

In [624]:
AT=pd.DataFrame(ATE,columns=['Cedula','Fecha2'])
AB=pd.DataFrame(ABA,columns=['FechaDia','Hora','Nit'])
IM=pd.DataFrame(impala,columns=['nro_identificacion','score_prioridad'])

In [625]:
AT.rename(columns={'Fecha2':'Fecha'},inplace=True)
AB.rename(columns={'FechaDia':'Fecha','Nit':'Cedula'},inplace=True)
IM.rename(columns={'nro_identificacion':'Cedula'},inplace=True)

In [626]:
AT['Abandonada']='No'
AB['Abandonada']='Si'
#base.assign(Abandonada='No')

In [627]:
AT.dtypes

Cedula                object
Fecha         datetime64[ns]
Abandonada            object
dtype: object

In [628]:
#AT[:,'Hora']= pd.datetime(hour=AT.Fecha.dt.hour, minute=AT.Fecha.dt.minute, second=AT.Fecha.dt.second)
AT['Hora']=AT['Fecha'].dt.hour.astype(str)+':'+AT['Fecha'].dt.minute.astype(str)+':'+AT['Fecha'].dt.second.astype(str)

In [629]:
AT

Cedula               Fecha Abandonada      Hora
0         38229171.0 2022-08-05 09:06:31         No    9:6:31
1       1118541589.0 2022-08-05 09:08:44         No    9:8:44
2                NaN 2022-08-05 09:10:01         No    9:10:1
3                NaN 2022-08-05 09:11:17         No   9:11:17
4          3776868.0 2022-08-05 09:34:14         No   9:34:14
...              ...                 ...        ...       ...
296334  1000579014.0 2022-08-31 23:45:50         No  23:45:50
296335    52084376.0 2022-08-31 23:51:13         No  23:51:13
296336     7959514.0 2022-08-31 23:52:55         No  23:52:55
296337    51838350.0 2022-08-31 23:57:12         No  23:57:12
296338  1014244813.0 2022-08-31 23:59:10         No  23:59:10

[296339 rows x 4 columns]

In [630]:
AB.loc[:,'Fecha'] = pd.to_datetime(AB.Fecha.astype(str)+' '+AB.Hora.astype(str))

In [631]:
from pandas import concat
CALL=concat( [AB,AT], axis=0)
CALL.index = range(CALL.shape[0])


In [632]:
CALL

Fecha      Hora        Cedula Abandonada
0      2022-08-01 06:25:16  06:25:16    79434991.0         Si
1      2022-08-01 06:29:17  06:29:17    41674527.0         Si
2      2022-08-01 06:39:02  06:39:02    86084655.0         Si
3      2022-08-01 06:42:14  06:42:14  1097388253.0         Si
4      2022-08-01 06:44:47  06:44:47    39439612.0         Si
...                    ...       ...           ...        ...
314123 2022-08-31 23:45:50  23:45:50  1000579014.0         No
314124 2022-08-31 23:51:13  23:51:13    52084376.0         No
314125 2022-08-31 23:52:55  23:52:55     7959514.0         No
314126 2022-08-31 23:57:12  23:57:12    51838350.0         No
314127 2022-08-31 23:59:10  23:59:10  1014244813.0         No

[314128 rows x 4 columns]

In [633]:
CALL.Cedula.isna().sum()

3447

### REEMPLAZA EL VALOR DE LA POSICIÓN j,i por -1

In [634]:
import re 
def extraer_tratamiento(name):
  m = re.search(r"\w+\.",str(name))
  if not m:
      return -1 # CON CARATERES *
  return m[0][:-1]

#### indexa en nueva columna

In [635]:
CALL['CedulaN']=CALL['Cedula'].apply(extraer_tratamiento)

In [636]:
CALL.dtypes

Fecha         datetime64[ns]
Hora                  object
Cedula                object
Abandonada            object
CedulaN               object
dtype: object

In [637]:
CALL['CedulaN']=CALL['CedulaN'].astype(np.int64)

In [638]:
CALL[CALL['CedulaN'] == -1]

Fecha      Hora  Cedula Abandonada  CedulaN
17791  2022-08-05 09:10:01    9:10:1     NaN         No       -1
17792  2022-08-05 09:11:17   9:11:17     NaN         No       -1
17794  2022-08-05 09:29:30   9:29:30     NaN         No       -1
17795  2022-08-05 09:38:20   9:38:20     NaN         No       -1
17797  2022-08-05 09:39:41   9:39:41     NaN         No       -1
...                    ...       ...     ...        ...      ...
313063 2022-08-31 18:51:06   18:51:6  TIENDA         No       -1
313600 2022-08-31 19:57:34  19:57:34  TIENDA         No       -1
313680 2022-08-31 20:09:01    20:9:1  TIENDA         No       -1
313681 2022-08-31 20:09:35   20:9:35  TIENDA         No       -1
313774 2022-08-31 20:27:29  20:27:29     NaN         No       -1

[5158 rows x 5 columns]

In [ ]:
#CALL[CALL['Cedula'].str.contains('*',regex=False)]

In [639]:
IM=pd.DataFrame(impala,columns=['nro_identificacion','score_prioridad'])
IM.rename(columns={'nro_identificacion':'CedulaN'},inplace=True)
#NAN
nueva_fila = {'CedulaN': np.nan, 'score_prioridad':0}
#SI TIENE CARACTER ESPECIAL COMO *, o algun caracter string 
nueva_fila2 = {'CedulaN':-1, 'score_prioridad':-1}

#Añadiendo una fila al dataframe
IM = IM.append(nueva_fila, ignore_index=True)
IM = IM.append(nueva_fila2, ignore_index=True)

C:\Users\miavila\AppData\Local\Temp\ipykernel_14348\1814763421.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  IM = IM.append(nueva_fila, ignore_index=True)
C:\Users\miavila\AppData\Local\Temp\ipykernel_14348\1814763421.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  IM = IM.append(nueva_fila2, ignore_index=True)


#### SIN CEDULA(VACIO)    CEDULA= NaN ,     SCORE= 0   3447
#### con caracteres esp CedulaN = -1,     SCORE= -1
#### celda vacia SCORE = no esta en impala=    SCORE=NaN    308601

In [640]:
IM.tail()

CedulaN  score_prioridad
19427   4170264.0              8.0
19428  78748361.0             12.0
19429  30030776.0              6.0
19430         NaN              0.0
19431        -1.0             -1.0

In [517]:
IM.columns,len(IM)

(Index(['CedulaN', 'score_prioridad'], dtype='object'), 19432)

# MERGE

In [646]:
CRUCE=pd.merge(CALL,IM,on=['CedulaN'], how='left')

In [647]:
CRUCE.score_prioridad.isna().sum()

308601

In [648]:
CRUCE

Fecha      Hora        Cedula Abandonada     CedulaN  \
0      2022-08-01 06:25:16  06:25:16    79434991.0         Si    79434991   
1      2022-08-01 06:29:17  06:29:17    41674527.0         Si    41674527   
2      2022-08-01 06:39:02  06:39:02    86084655.0         Si    86084655   
3      2022-08-01 06:42:14  06:42:14  1097388253.0         Si  1097388253   
4      2022-08-01 06:44:47  06:44:47    39439612.0         Si    39439612   
...                    ...       ...           ...        ...         ...   
314123 2022-08-31 23:45:50  23:45:50  1000579014.0         No  1000579014   
314124 2022-08-31 23:51:13  23:51:13    52084376.0         No    52084376   
314125 2022-08-31 23:52:55  23:52:55     7959514.0         No     7959514   
314126 2022-08-31 23:57:12  23:57:12    51838350.0         No    51838350   
314127 2022-08-31 23:59:10  23:59:10  1014244813.0         No  1014244813   

        score_prioridad  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN  
...                 ...  
314123              NaN  
314124              NaN  
314125              NaN  
314126              NaN  
314127              NaN  

[314128 rows x 6 columns]

In [649]:
CRUCE.score_prioridad.replace(np.nan,'NaN',inplace=True) #convierten a texto para que en excel aparezca

In [650]:
CRUCE.score_prioridad.isna().sum()

0

In [651]:
CRUCE.Cedula.replace(np.nan,'NaN',inplace=True)
CRUCE.CedulaN.replace(np.nan,'NaN',inplace=True)

In [652]:
CRUCE.rename(columns={'score_prioridad':'SCORE_PEP'},inplace=True)

In [653]:
CRUCE

Fecha      Hora        Cedula Abandonada     CedulaN  \
0      2022-08-01 06:25:16  06:25:16    79434991.0         Si    79434991   
1      2022-08-01 06:29:17  06:29:17    41674527.0         Si    41674527   
2      2022-08-01 06:39:02  06:39:02    86084655.0         Si    86084655   
3      2022-08-01 06:42:14  06:42:14  1097388253.0         Si  1097388253   
4      2022-08-01 06:44:47  06:44:47    39439612.0         Si    39439612   
...                    ...       ...           ...        ...         ...   
314123 2022-08-31 23:45:50  23:45:50  1000579014.0         No  1000579014   
314124 2022-08-31 23:51:13  23:51:13    52084376.0         No    52084376   
314125 2022-08-31 23:52:55  23:52:55     7959514.0         No     7959514   
314126 2022-08-31 23:57:12  23:57:12    51838350.0         No    51838350   
314127 2022-08-31 23:59:10  23:59:10  1014244813.0         No  1014244813   

       SCORE_PEP  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  
...          ...  
314123       NaN  
314124       NaN  
314125       NaN  
314126       NaN  
314127       NaN  

[314128 rows x 6 columns]

In [678]:
CRUCE.sort_values(by='Fecha') 
CRUCE.index = range(CRUCE.shape[0])

# los strings


In [679]:
CRUCE_NAN = CRUCE[CRUCE['Cedula'] == 'NaN']

In [680]:
CRUCE_NAN

Fecha      Hora Cedula Abandonada  CedulaN SCORE_PEP
17791  2022-08-05 09:10:01    9:10:1    NaN         No       -1      -1.0
17792  2022-08-05 09:11:17   9:11:17    NaN         No       -1      -1.0
17794  2022-08-05 09:29:30   9:29:30    NaN         No       -1      -1.0
17795  2022-08-05 09:38:20   9:38:20    NaN         No       -1      -1.0
17797  2022-08-05 09:39:41   9:39:41    NaN         No       -1      -1.0
...                    ...       ...    ...        ...      ...       ...
311929 2022-08-31 17:16:35  17:16:35    NaN         No       -1      -1.0
312121 2022-08-31 17:30:22  17:30:22    NaN         No       -1      -1.0
312193 2022-08-31 17:34:49  17:34:49    NaN         No       -1      -1.0
312374 2022-08-31 17:48:45  17:48:45    NaN         No       -1      -1.0
313774 2022-08-31 20:27:29  20:27:29    NaN         No       -1      -1.0

[3447 rows x 6 columns]

In [681]:
CRUCE_STRING = CRUCE.loc[(CRUCE['Cedula'] != 'NaN') & (CRUCE['SCORE_PEP'] == -1)]


In [682]:
CRUCE_STRING

Fecha      Hora  Cedula Abandonada  CedulaN SCORE_PEP
22329  2022-08-01 10:55:08   10:55:8  TIENDA         No       -1      -1.0
22505  2022-08-01 11:12:23  11:12:23  TIENDA         No       -1      -1.0
22732  2022-08-01 11:36:54  11:36:54  TIENDA         No       -1      -1.0
23446  2022-08-01 12:46:21  12:46:21  TIENDA         No       -1      -1.0
23455  2022-08-01 12:47:19  12:47:19  TIENDA         No       -1      -1.0
...                    ...       ...     ...        ...      ...       ...
313054 2022-08-31 18:50:13  18:50:13  TIENDA         No       -1      -1.0
313063 2022-08-31 18:51:06   18:51:6  TIENDA         No       -1      -1.0
313600 2022-08-31 19:57:34  19:57:34  TIENDA         No       -1      -1.0
313680 2022-08-31 20:09:01    20:9:1  TIENDA         No       -1      -1.0
313681 2022-08-31 20:09:35   20:9:35  TIENDA         No       -1      -1.0

[1711 rows x 6 columns]

In [498]:
#GUARDAR BASE
#CRUCE.to_excel('CRUCE ENGAGE.xlsx')

In [683]:
CRUCE2 = CRUCE.copy()
with pd.ExcelWriter('CRUCE ENGAGE.xlsx') as writer:  
    CRUCE2.to_excel(writer, sheet_name='CRUCE')
    CRUCE_NAN.to_excel(writer, sheet_name='CC NAN')
    CRUCE_STRING.to_excel(writer, sheet_name='CC MAL ESCRITAS')

# AGRUPACIÓN

In [684]:
CRUCE.columns
dfa=CRUCE

In [685]:
CRUCE.columns

Index(['Fecha', 'Hora', 'Cedula', 'Abandonada', 'CedulaN', 'SCORE_PEP'], dtype='object')

In [686]:
dfaa=dfa.assign(MesN=dfa.Fecha.dt.month ,Dia_MesN=dfa.Fecha.dt.date)

In [687]:
dfaa

Fecha      Hora        Cedula Abandonada     CedulaN  \
0      2022-08-01 06:25:16  06:25:16    79434991.0         Si    79434991   
1      2022-08-01 06:29:17  06:29:17    41674527.0         Si    41674527   
2      2022-08-01 06:39:02  06:39:02    86084655.0         Si    86084655   
3      2022-08-01 06:42:14  06:42:14  1097388253.0         Si  1097388253   
4      2022-08-01 06:44:47  06:44:47    39439612.0         Si    39439612   
...                    ...       ...           ...        ...         ...   
314123 2022-08-31 23:45:50  23:45:50  1000579014.0         No  1000579014   
314124 2022-08-31 23:51:13  23:51:13    52084376.0         No    52084376   
314125 2022-08-31 23:52:55  23:52:55     7959514.0         No     7959514   
314126 2022-08-31 23:57:12  23:57:12    51838350.0         No    51838350   
314127 2022-08-31 23:59:10  23:59:10  1014244813.0         No  1014244813   

       SCORE_PEP  MesN    Dia_MesN  
0            NaN     8  2022-08-01  
1            NaN     8  2022-08-01  
2            NaN     8  2022-08-01  
3            NaN     8  2022-08-01  
4            NaN     8  2022-08-01  
...          ...   ...         ...  
314123       NaN     8  2022-08-31  
314124       NaN     8  2022-08-31  
314125       NaN     8  2022-08-31  
314126       NaN     8  2022-08-31  
314127       NaN     8  2022-08-31  

[314128 rows x 8 columns]

In [702]:
agru = dfaa.groupby([dfaa.Dia_MesN, dfaa.SCORE_PEP,dfaa.Abandonada])['CedulaN'].count()
agruM= dfaa.groupby([dfaa.MesN, dfaa.SCORE_PEP,dfaa.Abandonada])['CedulaN'].count()

In [703]:
agru

Dia_MesN    SCORE_PEP  Abandonada
2022-08-01  -1.0       No              280
            4.0        No                2
            6.0        No                2
            7.0        No                1
            8.0        No                2
                                     ...  
2022-08-31  10.0       No                2
            11.0       No                9
            12.0       No                3
            NaN        No            13333
                       Si              564
Name: CedulaN, Length: 286, dtype: int64

In [704]:
agruM

MesN  SCORE_PEP  Abandonada
8     -1.0       No              5158
      1.0        No                 4
      3.0        No                 3
      4.0        No                22
                 Si                 2
      5.0        No                19
                 Si                 3
      6.0        No                42
                 Si                 3
      7.0        No                18
                 Si                 2
      8.0        No                34
                 Si                 6
      9.0        No                33
                 Si                 3
      10.0       No                13
      11.0       No                68
      12.0       No                72
      13.0       No                22
      NaN        No            290831
                 Si             17770
Name: CedulaN, dtype: int64

In [705]:
agru2 = agru.index.to_frame(name=['Dia_MesN','SCORE_PEP','Abandonada'])
agruMM=agruM.index.to_frame(name=['MesN','SCORE_PEP','Abandonada'])

In [706]:
agru2.index = range(agru2.shape[0])
agruMM.index= range(agruMM.shape[0])

In [707]:
DF=pd.DataFrame(agru2,columns=['Dia_MesN','SCORE_PEP','Abandonada'])
DFM=pd.DataFrame(agruMM,columns=['MesN','SCORE_PEP','Abandonada'])

In [708]:
recib=pd.DataFrame(agru.values,columns=['count_CC'])
recibM=pd.DataFrame(agruM.values,columns=['count_CC'])

In [709]:
CX=pd.concat([DF,recib], axis=1,)
CXM=pd.concat([DFM,recibM],axis=1)

In [711]:
CXM

MesN SCORE_PEP Abandonada  count_CC
0      8      -1.0         No      5158
1      8       1.0         No         4
2      8       3.0         No         3
3      8       4.0         No        22
4      8       4.0         Si         2
5      8       5.0         No        19
6      8       5.0         Si         3
7      8       6.0         No        42
8      8       6.0         Si         3
9      8       7.0         No        18
10     8       7.0         Si         2
11     8       8.0         No        34
12     8       8.0         Si         6
13     8       9.0         No        33
14     8       9.0         Si         3
15     8      10.0         No        13
16     8      11.0         No        68
17     8      12.0         No        72
18     8      13.0         No        22
19     8       NaN         No    290831
20     8       NaN         Si     17770

In [712]:
CX.to_excel('TABLE_SCORE_DIA.xlsx')

In [713]:
CXM.to_excel('Table_SCORE_MES.xlsx')

### GUARDA COMO .XLSX en el mismo link donde esta guardado este archivo .ipynb


## RESUMEN SI NO HUBIERA QUE HACER ETL

In [ ]:
CLV.rename(columns={'Cedula':'NRO_IDENTIFICACION'},inplace=True)
SCORE_P=pd.DataFrame(SCORE,columns=['NRO_IDENTIFICACION', 'SCORE_PRIORIDAD'])
df = pd.merge(CLV,SCORE_P,on=['NRO_IDENTIFICACION'],how='left')

In [ ]:
len(df)

67394

In [ ]:
df

Fecha  NRO_IDENTIFICACION Abandonada Plataforma  \
0     2021-12-16 00:00:00            201083.0         Si     Engage   
1     2021-12-16 00:00:00            477097.0         Si     Engage   
2     2021-12-16 00:00:00           4438398.0         Si     Engage   
3     2021-12-16 00:00:00           4538498.0         Si     Engage   
4     2021-12-16 00:00:00           7248970.0         Si     Engage   
...                   ...                 ...        ...        ...   
67389 2021-12-31 19:47:00           9865914.0         Si      Cloud   
67390 2021-12-31 19:50:00          72357854.0         No      Cloud   
67391 2021-12-31 19:55:00          19137341.0         Si      Cloud   
67392 2021-12-31 19:59:00          51947075.0         Si      Cloud   
67393 2021-12-31 19:59:00          53013847.0         Si      Cloud   

       SCORE_PRIORIDAD_x  SCORE_PRIORIDAD_y  SCORE_PRIORIDAD  
0                   10.0               10.0             10.0  
1                    NaN                NaN              NaN  
2                    NaN                NaN              NaN  
3                   10.0               10.0             10.0  
4                    NaN                NaN              NaN  
...                  ...                ...              ...  
67389               10.0               10.0             10.0  
67390               11.0               11.0             11.0  
67391               10.0               10.0             10.0  
67392               10.0               10.0             10.0  
67393                NaN                NaN              NaN  

[67394 rows x 7 columns]